In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt 

import torch

import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torchvision.transforms as transforms
from torch.utils.data import  Dataset, TensorDataset, DataLoader


import lib.pytorch_trainer as ptt

from src.data_loader import _create_dataLoader

from src.plot_utils import (plot_classes, plot_distribution,
                            statistical_analysis_image, classDistribution,
                            plot_metrics,visualize_predictions,
                            plot_cm_train_valid,plot_layers_weight)
      
from src.convnet_models import (MyResNet, MyInception, MyDenseNet)

from src.extractor_utils import (predict, getPrediction,features_loading)

from utils.utils import (create_submission ,metrics2csv, save_results)

In [2]:
print("{} GPU's available:".format(torch.cuda.device_count()) )
cpu_count = torch.multiprocessing.cpu_count()
print("\ncpu_count: {}".format(cpu_count))

4 GPU's available:

cpu_count: 24


In [3]:
use_gpu = False
use_DataParalel= True 
use_CPU= False        

if use_DataParalel: 
    print("Using DataParalel in all {} GPUS".format(torch.cuda.device_count()))

if use_gpu:
    print('Using only one GPU') 
    
if use_CPU:         # http://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html 
    print("Using {} CPU's".format(cpu_count))

Using DataParalel in all 4 GPUS


In [ ]:
if use_gpu:
    !free -h
    !nvidia-smi

# Path to features

In [4]:
path2features= "/mnt/home/e209440/state-farm-distracted-driver-detection/features/" 
#path2features= "/mnt/home/r120084/state-farm-distracted-driver-detection/features/" 

load_features = True

# Checking GPU usage

In [ ]:
!free -h
!nvidia-smi

# Choosing Model

In [ ]:
use_resnet = False
use_inception = False
use_denseNet = True

if use_resnet:
    print('Using ResNet model')
    model_name = "ResNet"
    model = MyResNet()
if use_inception:
    print('Using Inception model')
    model_name = "Inception"
    model = MyInception()
if use_denseNet:
    print('Using DenseNet model')
    model_name = "DenseNet"    
    model = MyDenseNet()

In [ ]:
if use_DataParalel:
    print("Using all GPU's ")
    model.mrnc = torch.nn.DataParallel(model.mrnc) #device_ids=[1,3]
    model.mrnc = model.mrnc.cuda()
    model.mrnd = torch.nn.DataParallel(model.mrnd) #device_ids=[1,3]
    model.mrnd = model.mrnd.cuda()

if use_gpu:
    print('Using GPU')# {}'.format(device_id))
    model.cuda()

if use_cpu:
    print("Using CPU's")


In [ ]:
#print(model)

# Loading features

In [ ]:
model_name

In [ ]:
load_feat= features_loading(path2features,model_name)

In [ ]:
load_feat['train'][0].shape,load_feat['train'][1].shape

In [ ]:
if load_features:
    conv_dset ={
    'train': TensorDataset(load_feat['train'][0], load_feat['train'][1]),
    'valid': TensorDataset(load_feat['valid'][0], load_feat['valid'][1]),
    'test': TensorDataset(load_feat['test'][0], load_feat['test'][1])
    }
else:
    conv_dset ={
    'train': TensorDataset(convOutput_train['train'][0], convOutput_train['train'][1]),
    'valid': TensorDataset(convOutput_valid['valid'][0], convOutput_valid['valid'][1]),
    'test': TensorDataset(convOutput_test['test'][0], convOutput_test['test'][1])
    }

In [ ]:
dset_loaders_convnet = _create_dataLoader(conv_dset, batch_size, 
                        pin_memory=False, use_shuffle= True)

In [ ]:
dset_convnet_sizes = {x: len(dset_loaders_convnet[x]) for x in ['train','valid', 'test']} 
dset_convnet_sizes

# Trainning model

In [ ]:
#path2saveModel = '/mnt/home/r120084/project/models/distractdriver'
path2saveModel = '/mnt/home/e209440/models/'+model_name+'DistractDriver' 

savebest = ptt.ModelCheckpoint(path2saveModel,reset=True, verbose=1)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
num_epochs = 70

optimizer =  optim.Adam(model.mrnd.parameters(), lr=1e-3)
scheduler = StepLR(optimizer, step_size=5, gamma=0.75)

params = {'model' : model.mrnd, 
    'criterion': loss_fn,  
    'optimizer': optimizer, 
    'callbacks': [savebest, ptt.AccuracyMetric(), ptt.PrintCallback()] #ptt.PlotCallback(),
}

In [ ]:
trainer = ptt.DeepNetTrainer(use_gpu=use_gpu,**params)

In [ ]:
trainer.fit_loader(num_epochs, dset_loaders_convnet['train'], dset_loaders_convnet['valid'])

# Loading best epoch

In [ ]:
trainer.load_state(path2saveModel)
model.mrnd = trainer.model

In [ ]:
!free -h
!nvidia-smi

In [ ]:
metrics2csv(trainer, 'ResNet18')

In [ ]:
train_eval = trainer.evaluate_loader(dset_loaders_convnet['train'])
valid_eval = trainer.evaluate_loader(dset_loaders_convnet['valid'])

In [ ]:
train_eval, valid_eval

# Evaluating model

In [ ]:
# need to create a dataloader without shuffle to compare the mismatch
dset_loaders_wshuffle = _create_dataLoader(dsets, batch_size, pin_memory=False, use_shuffle= False)

In [ ]:
#we need to use all dloader, because this one has use_shuffle false
result_train = predict(dset_loaders_wshuffle['train'], model, use_gpu=use_gpu)
result_valid = predict(dset_loaders_wshuffle['valid'], model, use_gpu=use_gpu)
result_test = predict(dset_loaders_wshuffle['test'], model, use_gpu=use_gpu)

In [ ]:
predictions_out = {'train': result_train, 'valid': result_valid, 'test': result_test}

In [ ]:
save_results(predictions_out, 'Resnet18')

In [ ]:
result_train = getPrediction(result_train)
result_valid = getPrediction(result_valid)
# result_test['pred'] must be an array of probabilities to make the submission

In [ ]:
correct_train = (result_train['true'] == result_train['pred']).sum()
correct_valid = (result_valid['true'] == result_valid['pred']).sum()

In [ ]:
print('Train: ', correct_train, '/', len(dsets['train']) )
print('Valid: ', correct_valid, '/', len(dsets['valid']) )

In [ ]:
visualize_predictions(dsets['valid'], result_valid, correct_pred = True)

In [ ]:
#move this to Data Visualization
#visualize_predictions(dsets['valid'], result_valid, correct_pred = False)

In [ ]:
#plot_cm_train_valid(result_train,result_valid)

In [ ]:
#move this to Data Visualization
#plot_layers_weight(dsets,img_width=img_width, img_height=img_height,conv_model = model.mrnc,use_gpu=use_gpu)

# Make submission of the Test set

In [ ]:
create_submission(result_test, 'distracted_driver')